In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("..\Corpora\ICE INDIA\Corpus\S1a-001.txt", sep="<$A>|<$B>", engine='python')


In [4]:
print(df)

                           <I>
0                         <$A>
1      <ICE-IND:S1A-001#1:1:A>
2    May I know your good name
3                         <$B>
4      <ICE-IND:S1A-001#2:1:B>
..                         ...
711  <ICE-IND:S1A-001#246:1:B>
712              Thank you <,>
713                       <$A>
714  <ICE-IND:S1A-001#247:1:A>
715   Thank you very much </I>

[716 rows x 1 columns]


In [5]:
df = df.replace(to_replace=r'<.*>', value='', regex=True)
print(df)

                           <I>
0                             
1                             
2    May I know your good name
3                             
4                             
..                         ...
711                           
712                 Thank you 
713                           
714                           
715       Thank you very much 

[716 rows x 1 columns]


In [6]:
padded_numbers = [number.zfill(3) for number in numbers]
print(padd_numbers)

NameError: name 'numbers' is not defined

In [6]:

count = [100, 80, 70, 50]
prefix = ["S1A", "S1B", "S2A", "S2B"]
for i in range(0, 4):
    end=count[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        combined_string=''
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE INDIA\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="<$A>|<$B>", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<.*>', value='', regex=True)
        #combined_string = ' '.join(temp_df.apply(lambda x: ' '.join(x), axis=1))
        new_filepath = "..\CleanCorpora\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False)
        
        
#print(combined_string)
    

In [15]:
count = [100, 80, 70, 50]
count2 = [1]
prefix = ["S1A", "S1B", "S2A", "S2B"]
for i in range(0, 1):
    end=count2[i]+1
    active_prefix = prefix[i]
    for i in range(1, end):
        combined_string=''
        #
        index_padded = str(i).zfill(3)
        filepath = "..\Corpora\ICE INDIA\Corpus\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df = pd.read_csv(filepath, sep="\t", engine='python', encoding='ISO-8859-1')
        temp_df = temp_df.replace(to_replace=r'<.*>', value='', regex=True)
        new_filepath = "..\CleanCorpora\{0}-{1}.txt".format(active_prefix, index_padded)
        temp_df.to_csv(new_filepath, sep='\t', index=False)
        print(combined_string)
        print(combined_string2)

AttributeError: 'DataFrame' object has no attribute 'str'

['S1A', 'S1B', 'S2A', 'S2B']